In [1]:
import yaml
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd

# 1.  Read the csv file using python

In [ ]:
# Primero se carga el archivo csv como un dataframe
file_csv = pd.read_csv("candidates.csv", sep=";")
print(file_csv.head())

   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

In [5]:
print(file_csv["First Name"])

0        Bernadette
1            Camryn
2             Larue
3              Arch
4             Larue
            ...    
49995       Bethany
49996           Era
49997        Martin
49998         Aliya
49999       Coleman
Name: First Name, Length: 50000, dtype: object


# 2.  load the data into the database (staging area)

In [ ]:
# Cargamos datos de configuración de la base de datos
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [ ]:
# Realizamos la conexión a la base de datos postgres
config = load_config()
db_config = config["database"]

# Load credentials
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

print(f"Usuario: {db_user}, Host: {db_host}, Puerto: {db_port}, Base de Datos: {db_name}")

# DB connection

conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True
print("Conexión exitosa")

Usuario: postgres, Host: localhost, Puerto: 5432, Base de Datos: workshop1_db
Conexión exitosa


In [ ]:
# Creamos la base de datos para el ejercicio
db_name = "workshopl_db"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close()

Base de datos 'workshopl_db' creada exitosamente.


In [ ]:
# Creamos la tabla con las columnas correspondientes
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS tabla_etl (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(100),
            application_date TIMESTAMP,
            country VARCHAR(100),
            yoe INT,
            seniority VARCHAR(100),
            technology VARCHAR(100),
            code_challenge_score INT,
            technical_interview_score INT,
            fecha_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """))
    conn.commit()
    print("Tabla 'workshopl_db' creada exitosamente en PostgreSQL.")

Tabla 'workshopl_db' creada exitosamente en PostgreSQL.


In [ ]:
# Como el nombre de las columnas en el dataframe difiere del nombre de las columnas en db, renombramos las columnas del dataframe para facilitar el cargue de datos desde python
rename_columnas = {
    "First Name": "first_name",
    "Last Name": "last_name",
    "Email": "email",
    "Application Date": "application_date",
    "Country": "country",
    "YOE": "yoe",
    "Seniority": "seniority",
    "Technology": "technology",
    "Code Challenge Score": "code_challenge_score",
    "Technical Interview Score": "technical_interview_score"
}

file_csv.rename(columns=rename_columnas, inplace=True)

first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(100),
            application_date TIMESTAMP,
            country VARCHAR(100),
            yoe INT,
            seniority VARCHAR(100),
            technology VARCHAR(100),
            code_challenge_score INT,
            technical_interview_score INT,

In [ ]:
# Observamos el resultado obtenido
file_csv

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


In [ ]:
# Subimos los datos a la base de datos usando una función de pandas
file_csv.to_sql("workshopl_db", engine, if_exists="append", index=False)

1000

# 3. read the table you just created with python

In [ ]:
# Leer datos de la tabla staging
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM workshopl_db;"))
    rows = result.fetchall()

In [ ]:
# Leer datos de la tabla staging
with engine.connect() as conn:
    data_transform = pd.read_sql("SELECT * FROM workshopl_db", conn)

In [15]:
data_transform

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


# 4. do the transformations named bellow

In [ ]:
# Para el ejercicio sacamos una copia del dataframe original resultando de la extracción desde el staging para hacer las transformaciones
df_transformed = data_transform.copy()

In [ ]:
# Aplicamos la lógica del ejercicio, asignar el valor "HIRED" si ambos score son mayores o iguales a 7, de lo contrario "NO HIRED"
df_transformed["hired"] = df_transformed.apply(lambda row: "HIRED" if row["code_challenge_score"] >= 7 and row["technical_interview_score"] >= 7 else "NO HIRED", axis=1)

In [ ]:
# Visualizamos el resultado con la nueva columna
df_transformed

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,NO HIRED
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,NO HIRED
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,HIRED
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,NO HIRED
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,HIRED
...,...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1,NO HIRED
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2,NO HIRED
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1,NO HIRED
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0,NO HIRED


#  5. load the transformed data into a new table

In [ ]:
# Creamos una nueva tabla incluyendo la nueva columna
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS transformed_data (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(100),
            application_date TIMESTAMP,
            country VARCHAR(100),
            yoe INT,
            seniority VARCHAR(100),
            technology VARCHAR(100),
            code_challenge_score INT,
            technical_interview_score INT,
            hired VARCHAR(100),
            fecha_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """))
    conn.commit()
    print("Tabla 'transformed_data' creada exitosamente en PostgreSQL.")

Tabla 'transformed_data' creada exitosamente en PostgreSQL.


In [ ]:
# Almacenamos los datos transformados en la nueva tabla
with engine.connect() as conn:
    df_transformed.to_sql("transformed_etl", con=engine, if_exists="append", index=False)

print("Transformed data stored successfully in 'transformed_etl'.")

Transformed data stored successfully in 'transformed_etl'.


In [ ]:
# Realizamos una lectura de la nueva tabla para corroborar los datos almacenados con transformación
with engine.connect() as conn:
    db_transformed_df = pd.read_sql("SELECT * FROM transformed_etl", conn)
    
db_transformed_df

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,NO HIRED
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,NO HIRED
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,HIRED
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,NO HIRED
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,HIRED
...,...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1,NO HIRED
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2,NO HIRED
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1,NO HIRED
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0,NO HIRED
